In [9]:
# !wget https://raw.githubusercontent.com/DataTalksClub/llm-zoomcamp/refs/heads/main/01-intro/minsearch.py
# !wget https://raw.githubusercontent.com/DataTalksClub/llm-zoomcamp/refs/heads/main/01-intro/documents.json

In [2]:
from openai import OpenAI
from dotenv import load_dotenv
import os
import minsearch
import json

In [8]:
with open("documents.json", 'rt') as f_in:
    docs_raw = json.load(f_in)

documents = []
for course in docs_raw:
    for doc in course['documents']:
        doc['course'] = course['course']
        documents.append(doc)

In [9]:
index = minsearch.Index(
    text_fields=["question", "text", "section"]
    ,keyword_fields=["course"]
)

index.fit(documents)

In [13]:
q = "the course has alraedy started, can I still enroll?"

boost = {"question": 3.0, "section": 0.5}

results = index.search(q
                       , boost_dict=boost
                       , num_results=5
                       , filter_dict={"course": "data-engineering-zoomcamp"})

results                       

[{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.',
  'section': 'General course-related questions',
  'question': 'Course - Can I follow the course after it finishes?',
  'course': 'data-engineering-zoomcamp'},
 {'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 202

In [31]:
load_dotenv(".env")

True

In [32]:
client = OpenAI()

In [ ]:
def search(query: str) -> list:
    boost = {"question": 3.0, "section": 0.5}
    results = index.search(
        query
        , boost_dict=boost
        , num_results=5
        , filter_dict={"course": "data-engineering-zoomcamp"}
    ) 
    
    return results

[{'text': 'In the project directory, run:\njava -cp build/libs/<jar_name>-1.0-SNAPSHOT.jar:out src/main/java/org/example/JsonProducer.java',
  'section': 'Module 6: streaming with kafka',
  'question': 'Java Kafka: How to run producer/consumer/kstreams/etc in terminal',
  'course': 'data-engineering-zoomcamp'},
 {'text': "Solution from Alexey: create a virtual environment and run requirements.txt and the python files in that environment.\nTo create a virtual env and install packages (run only once)\npython -m venv env\nsource env/bin/activate\npip install -r ../requirements.txt\nTo activate it (you'll need to run it every time you need the virtual env):\nsource env/bin/activate\nTo deactivate it:\ndeactivate\nThis works on MacOS, Linux and Windows - but for Windows the path is slightly different (it's env/Scripts/activate)\nAlso the virtual environment should be created only to run the python file. Docker images should first all be up and running.",
  'section': 'Module 6: streaming wi

In [82]:
def build_prompt(question: str, search_results: list) -> str:
    
    question = question.strip()

    # Prompt template format
    prompt_template = """
    You're a course teaching assistant. Answer the QUESTION based on the CONTEXT, using only facts from the CONTEXT. 

    QUESTION: {question}
    CONTEXT: 
    {context}
    """.strip()

    # build the context string 
    context = ""
    
    for doc in search_results:
        context += f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"
    
    prompt = prompt_template.format(question=question, context=context)
    return prompt



In [83]:
def llm(prompt: str) -> str:
    response=client.chat.completions.create(
        model='gpt-4o'
        ,messages=[{'role': 'user', 'content': prompt}]
        )
    # print(prompt)
    return response.choices[0].message.content

In [ ]:
query = "How do I run Kafka?"
def rag(query: str) -> str:
    search_results = search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer   


'To run Kafka, you can use the following command in the project directory:\n\n```bash\njava -cp build/libs/<jar_name>-1.0-SNAPSHOT.jar:out src/main/java/org/example/JsonProducer.java\n```'